In [7]:
import joblib
import numpy as np
import pandas as pd


In [8]:
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



In [9]:
ruta='datos/'
df_train_three=joblib.load(ruta+'df_train_datos2.pkl')
df_xtest_three=joblib.load(ruta+'df_xtest_datos2.pkl')



X=df_train_three.loc[:,~df_train_three.columns.isin(['target_duo', 'target_trio'])]
X_real_test=df_xtest_three.loc[:,~df_xtest_three.columns.isin(['target_duo', 'target_trio'])]
y2=df_train_three.loc[:,df_train_three.columns.isin(['target_duo'])]
y3=df_train_three.loc[:,df_train_three.columns.isin(['target_trio'])]


test_size=0.8
random_state=1000
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, \
    test_size=test_size, random_state=random_state)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y3, \
    test_size=test_size, random_state=random_state)

In [10]:

log_reg = LogisticRegression(random_state=random_state)
knn = KNeighborsClassifier()
svm = SVC(random_state=random_state)
mlp = MLPClassifier(max_iter=500,random_state=random_state)
rf = RandomForestClassifier(random_state=random_state)
dt=DecisionTreeClassifier(random_state=random_state)
xgb = XGBClassifier(random_state=random_state, verbosity=0)

log_reg_params = {'classifier__C': [0.1, 1, 10]}

xgb_params = {
    'classifier__learning_rate': [0.05, 0.1, 0.2],
    'classifier__max_depth': [1, 2, 3, 5, 8],
    'classifier__reg_lambda': [0, 1, 10],
}

nb_params={
    'classifier__var_smoothing': np.linspace(-9,0,num=10).astype(int)
}

knn_params={
    "classifier__n_neighbors":[3,5,7,9,11],
    "classifier__weights":['uniform','distance']
}

# no se usa, demasiado lento
mlp_params = {
    'classifier__hidden_layer_sizes': [(150,100,50), (100,50,30)],
    'classifier__activation': ['tanh', 'relu'],
    'classifier__solver': ['sgd', 'adam'],
    'classifier__alpha': [0.0001, 0.05],
}


dt_params={}

rf_params={
    'classifier__n_estimators':np.linspace(100,900,9).astype(int),
    'classifier__criterion':['gini', 'entropy'],
    'classifier__min_samples_leaf': [1, 2, 3, 4],
}

svm_params={
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'classifier__kernel': ['rbf']
}

clfs = [
        ('Logistic Regression', log_reg,log_reg_params), 
        ('K-Nearest Neighbors', knn,knn_params), 
        ('SVM', svm,svm_params), 
        #('MLP', mlp,mlp_params), 
        ('Decission Tree', dt,dt_params),
        ('Random Forest', rf,rf_params), 
        ('XGBoost', xgb,xgb_params)
        ]



In [11]:
pipelines=[]


scores_df = pd.DataFrame(columns=['Model', 'F1_Score', 'Precision', 'Recall', \
    'Accuracy','params'])


for clf_name, clf, parametros in clfs:
    print(clf_name)
    print(clf)
    print(parametros)
    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', clf),
    ])
    pipeline = GridSearchCV(pipeline,verbose=3, param_grid=parametros, refit=True)
    pipeline.fit(X_train3, y_train3.values.ravel())
    pipelines.append(pipeline)

    print(f"fit realizado con clasificador {clf_name}")

    y_pred3 = pipeline.predict(X_test3)
    # F1-Score
    fscore = f1_score(y_test3, y_pred3, average='macro')
    # Precision
    pres = precision_score(y_test3, y_pred3, average='macro')
    # Recall
    rcall = recall_score(y_test3, y_pred3, average='macro')
    # Accuracy
    accu = accuracy_score(y_test3, y_pred3)

    pipelines.append(pipeline)

    scores_df = scores_df.append({'Model' : clf_name, 
                                    'F1_Score' : fscore,
                                    'Precision' : pres,
                                    'Recall' : rcall,
                                    'Accuracy' : accu,
                                    'params':str(pipeline.best_params_)
                                    }, 
                                    ignore_index=True)


Logistic Regression
LogisticRegression(random_state=1000)
{'classifier__C': [0.1, 1, 10]}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END .................classifier__C=0.1;, score=0.699 total time=   0.1s
[CV 2/5] END .................classifier__C=0.1;, score=0.717 total time=   0.0s
[CV 3/5] END .................classifier__C=0.1;, score=0.708 total time=   0.0s
[CV 4/5] END .................classifier__C=0.1;, score=0.707 total time=   0.0s
[CV 5/5] END .................classifier__C=0.1;, score=0.716 total time=   0.0s
[CV 1/5] END ...................classifier__C=1;, score=0.699 total time=   0.1s
[CV 2/5] END ...................classifier__C=1;, score=0.717 total time=   0.0s
[CV 3/5] END ...................classifier__C=1;, score=0.708 total time=   0.0s
[CV 4/5] END ...................classifier__C=1;, score=0.707 total time=   0.0s
[CV 5/5] END ...................classifier__C=1;, score=0.716 total time=   0.1s
[CV 1/5] END ..................classifie

C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5] END classifier__activation=tanh, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(150, 100, 50), classifier__solver=sgd;, score=0.727 total time= 1.9min


C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END classifier__activation=tanh, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(150, 100, 50), classifier__solver=sgd;, score=0.723 total time= 1.8min


...

[CV 4/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=sgd;, score=0.721 total time= 1.4min


C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 5/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=sgd;, score=0.732 total time= 1.4min
[CV 1/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=adam;, score=0.722 total time=  40.6s
[CV 2/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=adam;, score=0.735 total time=  40.7s
[CV 3/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=adam;, score=0.734 total time=  26.6s
[CV 4/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=adam;, score=0.726 total time=  37.0s
[CV 5/5] END classifier__activation=relu, classifier__alpha=0.05, classifier__hidden_layer_sizes=(100, 50, 30), classifier__solver=adam;, score=0.726 total 

C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


fit realizado con clasificador MLP
Decission Tree
DecisionTreeClassifier(random_state=1000)
{}
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..................................., score=0.678 total time=   0.0s
[CV 2/5] END ..................................., score=0.687 total time=   0.0s
[CV 3/5] END ..................................., score=0.686 total time=   0.0s
[CV 4/5] END ..................................., score=0.691 total time=   0.0s
[CV 5/5] END ..................................., score=0.686 total time=   0.0s
fit realizado con clasificador Decission Tree
Random Forest
RandomForestClassifier(random_state=1000)
{'classifier__n_estimators': array([100, 200, 300, 400, 500, 600, 700, 800, 900])}
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ......classifier__n_estimators=100;, score=0.756 total time=   1.0s
[CV 2/5] END ......classifier__n_estimators=100;, score=0.763 total time=   1.0s
[CV 3/5] END ......classifier__n_e

In [15]:
scores_df

,Model,F1_Score,Precision,Recall,Accuracy,params
0,Logistic Regression,0.478264,0.488719,0.490823,0.711237,{'classifier__C': 10}
1,K-Nearest Neighbors,0.589644,0.635390,0.573425,0.738931,"{'classifier__n_neighbors': 11, 'classifier__w..."
2,SVM,0.547913,0.647551,0.536868,0.731439,"{'classifier__C': 10, 'classifier__gamma': 0.1..."
3,MLP,0.546653,0.646184,0.537748,0.732050,"{'classifier__activation': 'relu', 'classifier..."
4,Decission Tree,0.573161,0.571794,0.574657,0.694318,{}
5,Random Forest,0.621563,0.679930,0.601135,0.764752,{'classifier__n_estimators': 700}
6,XGBoost,0.585928,0.696757,0.566549,0.753472,"{'classifier__learning_rate': 0.1, 'classifier..."


In [20]:
pipelines[indice_mejor.values[0]]

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC(random_state=1000))]),
             param_grid={'classifier__C': [0.1, 1, 10, 100],
                         'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'classifier__kernel': ['rbf']},
             verbose=3)

In [17]:
# Preparamos salida.
indice_mejor=scores_df[['Accuracy']].idxmax()
print(f"mejor resultado en {scores_df['Model'][indice_mejor]}")
salida_prediccion=pipelines[indice_mejor.values[0]].predict(X_real_test)

mejor resultado en 5    Random Forest
Name: Model, dtype: object


In [18]:
from pathlib import Path

df_xtest_three["respuesta"]=salida_prediccion
df_xtest_three["status_group"]=df_xtest_three["respuesta"].replace([0, 1,2],['functional', 'functional needs repair','non functional'])

ruta='datos/'
nom='respuesta'
num=3
converted_num = "{}".format(num)
file=ruta+nom+converted_num+'.csv'
escrito=False
num=1
while escrito==False:
    converted_num = "{}".format(num)
    file=ruta+nom+converted_num+'.csv'
    path=Path(file)
    if path.is_file():
        num=num+1
        if num==100:
            escrito=False
            print("algo falla en el bucle")
    else:
        print(f"Aquí escribiría {file}")
        #df_xtest_three['status_group'].to_csv(file) 
        escrito=True






Aquí escribiría datos/respuesta4.csv


In [23]:
# joblib.dump(pipelines,'results/pipes1.z')
# joblib.dump(pipelines,'results/scores_df_pipes1.z')

['results/scores_df_pipes1.z']